# IBS of 3 bars - Strategy - Backtesting 

### Import Library

In [102]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [103]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [104]:
data = dataset.copy()

In [105]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [106]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524
2024-11-25 14:30:00,1297.1,1297.1,1297.1,1297.1,161
2024-11-25 14:45:00,1298.4,1298.4,1298.4,1298.4,5627


In [107]:
def set_condition_1(r):
    cond = ''
    if r['prev_2_Open'] < r['prev_2_Close']:
        # cay nen thu 2 truoc cay nen hien tai la cay nen tang
        cond = 'short'
    elif r['prev_2_Open'] > r['prev_2_Close']:
        # cay nen thu 2 truoc cay nen hien tai la cay nen giam
        cond = 'long'
    return cond

def set_condition_2(r):
    cond = ''
    if r['Open'] > r['Close']:
        # cay nen hien tai la cay nen giam
        cond = 'short'
    elif r['Open'] < r['Close']:
        # cay nen hien tai la cay nen tang
        cond = 'long'
    return cond

def set_condition_3(r):
    cond = ''
    if r['Close'] > r['prev_2_Open']:
        # close cua nen hien tai > open cua cay nen thu 2 truoc do
        cond = 'long'
    elif r['Close'] < r['prev_2_Open']:
        # close cua nen hien tai < open cua cay nen thu 2 truoc do
        cond = 'short'
    return cond

def set_condition_4(r):
    cond = ''
    if r['sum_ibs'] > 1.7:
        cond = 'long'
    return cond

def get_signal(r):
    signal = ''
    if r['condition_1'] == 'short' and r['condition_2'] == 'short' and r['condition_3'] == 'short' and r['condition_4'] == 'short':
        signal = 'short'
    elif r['condition_1'] == 'long' and r['condition_2'] == 'long' and r['condition_3'] == 'long' and r['condition_4'] == 'long':
        signal = 'long'
    return signal

In [108]:
def prepare_data(data):
    data['max_5'] = data['High'].rolling(5).max()
    data['min_5'] = data['Low'].rolling(5).min()
    data['prev_2_Open'] = data['Open'].shift(2)
    data['prev_2_Close'] = data['Close'].shift(2)
    data['ibs'] = data.apply(lambda x: (-1.0 if (x["High"] == x["Low"]) else (x["Close"] - x["Low"]) / (x["High"] - x["Low"])), axis=1)
    data['sum_ibs'] = data['ibs'].rolling(3).sum()
    data['condition_1'] = data.apply(lambda r: set_condition_1(r), axis=1)
    data['condition_2'] = data.apply(lambda r: set_condition_2(r), axis=1)
    data['condition_3'] = data.apply(lambda r: set_condition_3(r), axis=1)
    data['condition_4'] = data.apply(lambda r: set_condition_4(r), axis=1)
    data['signal'] = data.apply(lambda r: get_signal(r), axis=1)
    return data

In [109]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [110]:
prepared_data

,Open,High,Low,Close,Volume,max_5,min_5,prev_2_Open,prev_2_Close,ibs,sum_ibs,condition_1,condition_2,condition_3,condition_4,signal
Date,,,,,,,,,,,,,,,,
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939,900.2,896.5,899.5,899.5,0.809524,1.617216,,short,short,,
2020-11-02 09:25:00,898.3,898.6,897.8,898.5,1607,900.2,896.5,899.4,898.6,0.875000,1.992216,long,long,short,long,
2020-11-02 09:30:00,898.5,899.4,898.3,899.0,1623,900.0,896.5,898.5,898.2,0.636364,2.320887,long,long,long,long,long
2020-11-02 09:35:00,899.1,899.8,898.7,898.9,2180,899.8,896.5,898.3,898.5,0.181818,1.693182,short,short,long,,
2020-11-02 09:40:00,899.0,899.4,898.7,898.8,1190,899.8,896.5,898.5,899.0,0.142857,0.961039,short,short,long,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524,1298.7,1295.9,1296.7,1297.5,0.187500,1.531029,short,short,long,,
2024-11-25 14:30:00,1297.1,1297.1,1297.1,1297.1,161,1298.7,1295.9,1297.1,1298.0,-1.000000,0.011029,short,,,,
2024-11-25 14:45:00,1298.4,1298.4,1298.4,1298.4,5627,1298.7,1296.2,1298.0,1297.0,-1.000000,-1.812500,long,,long,,


In [111]:
prepared_data[prepared_data.signal != '']

,Open,High,Low,Close,Volume,max_5,min_5,prev_2_Open,prev_2_Close,ibs,sum_ibs,condition_1,condition_2,condition_3,condition_4,signal
Date,,,,,,,,,,,,,,,,
2020-11-02 09:30:00,898.5,899.4,898.3,899.0,1623,900.0,896.5,898.5,898.2,0.636364,2.320887,long,long,long,long,long
2020-11-02 09:55:00,898.7,901.4,898.6,901.3,3705,901.4,897.7,898.8,898.1,0.964286,1.712537,long,long,long,long,long
2020-11-02 13:35:00,898.3,899.5,898.0,899.1,3239,899.5,896.3,897.4,896.7,0.733333,1.820635,long,long,long,long,long
2020-11-02 13:50:00,900.4,903.0,900.4,902.5,7061,903.0,896.6,899.2,897.7,0.807692,2.125874,long,long,long,long,long
2020-11-02 14:20:00,899.0,905.5,899.0,904.5,7646,905.5,896.6,898.0,897.7,0.846154,2.187006,long,long,long,long,long
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-22 10:15:00,1295.0,1296.2,1295.0,1295.9,2840,1296.2,1293.3,1294.7,1294.4,0.750000,2.321429,long,long,long,long,long
2024-11-22 10:45:00,1292.9,1294.8,1292.9,1294.5,3971,1294.8,1291.8,1293.0,1292.7,0.842105,2.242105,long,long,long,long,long
2024-11-22 11:10:00,1296.0,1297.5,1295.9,1296.9,5263,1297.5,1294.0,1295.6,1294.9,0.625000,1.832692,long,long,long,long,long


In [113]:
class Ibs3BarsStrategy(Strategy):
    trailing_sl = 3.5
    def init(self):
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
                return

        close_price = self.data.Close[-1]
        if self.position.is_long:
            max_5 = self.data.max_5[-1]
            if close_price < max_5 - self.trailing_sl:
                self.position.close()
        elif self.position.is_short:
            min_5 = self.data.min_5[-1]
            if close_price > min_5 + self.trailing_sl:
                self.position.close()

        if not self.position:
            signal = self.data.signal[-1]
            if signal == 'long':
                buy_price = close_price
                sl = buy_price - 3
                tp = buy_price + 6
                self.buy(size=1, sl=sl, tp=tp)
            elif signal == 'short':
                sell_price = close_price
                sl = sell_price + 3
                tp = sell_price - 6
                self.sell(size=1, sl=sl, tp=tp)

In [114]:
bt = Backtest(prepared_data, Ibs3BarsStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [115]:
stats

Start                     2020-11-02 09:20:00
End                       2024-11-26 09:05:00
Duration                   1484 days 23:45:00
Exposure Time [%]                   34.636774
Equity Final [$]                   9452.83442
Equity Peak [$]                   10065.54756
Return [%]                          -5.471656
Buy & Hold Return [%]               44.533511
Return (Ann.) [%]                   -1.376544
Volatility (Ann.) [%]                0.913695
Sharpe Ratio                        -1.506568
Sortino Ratio                       -2.019631
Calmar Ratio                        -0.220461
Max. Drawdown [%]                   -6.243944
Avg. Drawdown [%]                   -0.238026
Max. Drawdown Duration     1259 days 23:30:00
Avg. Drawdown Duration       39 days 19:23:00
# Trades                                 2171
Win Rate [%]                        36.526946
Best Trade [%]                       0.845233
Worst Trade [%]                     -0.931468
Avg. Trade [%]                    

In [116]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,1,3,20,899.36973,896.0,-3.36973,-0.003747,2020-11-02 09:35:00,2020-11-02 11:00:00,None,0 days 01:25:00
1,1,35,41,899.46976,898.0,-1.46976,-0.001634,2020-11-02 13:40:00,2020-11-02 14:10:00,None,0 days 00:30:00
2,1,44,45,905.17147,906.7,1.52853,0.001689,2020-11-02 14:25:00,2020-11-02 14:30:00,None,0 days 00:05:00
3,1,55,81,908.87258,905.8,-3.07258,-0.003381,2020-11-03 09:40:00,2020-11-03 13:15:00,None,0 days 03:35:00
4,1,84,88,907.67222,904.4,-3.27222,-0.003605,2020-11-03 13:30:00,2020-11-03 13:50:00,None,0 days 00:20:00
...,...,...,...,...,...,...,...,...,...,...,...
2166,1,51668,51678,1276.58286,1282.1,5.51714,0.004322,2024-11-21 13:30:00,2024-11-21 14:20:00,None,0 days 00:50:00
2167,1,51698,51699,1296.28877,1292.9,-3.38877,-0.002614,2024-11-22 10:20:00,2024-11-22 10:25:00,None,0 days 00:05:00
2168,1,51704,51717,1294.88835,1300.5,5.61165,0.004334,2024-11-22 10:50:00,2024-11-22 13:20:00,None,0 days 02:30:00
2169,1,51722,51722,1300.39000,1296.8,-3.59000,-0.002761,2024-11-22 13:45:00,2024-11-22 13:45:00,None,0 days 00:00:00
